<a href="https://www.kaggle.com/code/vtrackstar/hooper-island-fantasy-basketball?scriptVersionId=270267354" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install espn-api

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from espn_api.basketball import League

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
google-cloud-bigtable 2.32.0 requires google-api-core[grpc]<3.0.0,>=2.17.0, but you have google-api-core 1.34.1 which is incompatible.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.40.3 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is in

In [2]:
# League Info
league_id = 2072742117
season_id = 2026
swid = '{A5D0418F-EB37-4A5B-9041-8FEB372A5BCC}'
espn_s2 = 'AECpVxhDRvfrKeWJdI9zJkUUbpoBPS5ybsG8SLFsL%2BIB%2ByM0zKt0xT2qU%2Bhoa%2B05NvvO%2BSkz3SK7VkToD4Yw5fg89Vrj8qPB%2FGHC3W81lJ9bWPTUw3Ss9YSFuYUIkorKnXJBs3YefGPPo4n9SabSdlY0YbEx2oQ%2Bdls8e3JcKMVHXf4j9ER3v1qY406MEWmlIc0Ubw2u7cEjs1yUGWlOPfMcZml%2BTUkTVF6HJhEjzWQ6bhOGUHEif8KfpJ4TY1ydkf8l%2F5JycpD9m1cQ8bP9f4g5b5EqmF4U0ZxHrM6FFc%2FPSw%3D%3D'

league = League(league_id=league_id, year=season_id, espn_s2=espn_s2, swid=swid)

In [3]:
print("League Name:", league.settings.name)
for team in league.teams:
    print(team.team_name, "-", [p.name for p in team.roster])

League Name: My 2024 League
Geriatric Knee's and Buckets - ['Cade Cunningham', 'Josh Giddey', 'Kevin Durant', 'Chet Holmgren', 'Jaren Jackson Jr.', 'Jimmy Butler III', 'Draymond Green', 'Kawhi Leonard', 'RJ Barrett', "Kel'el Ware", 'Cam Thomas', 'Ausar Thompson', 'Christian Braun']
TytheGreat - ['Donovan Mitchell', 'Bam Adebayo', 'Jalen Williams', 'Dyson Daniels', 'Miles Bridges', 'Anfernee Simons', 'Ivica Zubac', 'Zach LaVine', 'Julius Randle', "De'Aaron Fox", 'Jaden McDaniels', 'Paul George', 'Immanuel Quickley', 'VJ Edgecombe']
Da Man Dem - ['Giannis Antetokounmpo', 'Amen Thompson', 'James Harden', 'Evan Mobley', 'Norman Powell', 'Jordan Poole', 'Zion Williamson', 'DeMar DeRozan', 'Payton Pritchard', 'Deandre Ayton', 'Andrew Nembhard', 'Ace Bailey', 'Reed Sheppard']
Real Njggas do Real Things - ['Anthony Edwards', 'Stephen Curry', 'Karl-Anthony Towns', 'Jaylen Brown', 'Desmond Bane', 'Cooper Flagg', 'Nikola Vucevic', 'Trey Murphy III', 'Brandon Miller', 'Kevin Porter Jr.', 'CJ McCol

In [4]:
team_list = []
player_list = []

for team in league.teams:
    for player in team.roster:
        team_list.append(team.team_name)
        player_list.append(player.name)

fantasydf = pd.DataFrame({
    "Team": team_list,
    "Player": player_list
})

fantasydf

,Team,Player
0,Geriatric Knee's and Buckets,Cade Cunningham
1,Geriatric Knee's and Buckets,Josh Giddey
2,Geriatric Knee's and Buckets,Kevin Durant
3,Geriatric Knee's and Buckets,Chet Holmgren
4,Geriatric Knee's and Buckets,Jaren Jackson Jr.
...,...,...
103,Dev,D'Angelo Russell
104,Dev,Nic Claxton
105,Dev,Collin Sexton
106,Dev,Davion Mitchell


In [5]:
url = "https://www.basketball-reference.com/leagues/NBA_2026_totals.html#totals_stats"
tables = pd.read_html(url)
season_stats = tables[0]
print(season_stats.head(), season_stats.columns)

    Rk             Player   Age Team Pos    G   GS    MP    FG   FGA  ...  \
0  1.0        Luka Dončić  26.0  LAL  PG  1.0  1.0  41.0  17.0  27.0  ...   
1  2.0    Anthony Edwards  24.0  MIN  SG  1.0  1.0  39.0  14.0  28.0  ...   
2  3.0       Tyrese Maxey  25.0  PHI  PG  1.0  1.0  41.0  13.0  24.0  ...   
3  4.0  Victor Wembanyama  22.0  SAS   C  1.0  1.0  30.0  15.0  21.0  ...   
4  5.0     Alperen Şengün  23.0  HOU   C  1.0  1.0  49.0  12.0  24.0  ...   

    DRB   TRB  AST  STL  BLK  TOV   PF   PTS  Trp-Dbl  Awards  
0  12.0  12.0  9.0  2.0  1.0  3.0  1.0  43.0      0.0     NaN  
1   6.0   7.0  1.0  0.0  1.0  4.0  4.0  41.0      0.0     NaN  
2   2.0   2.0  6.0  2.0  1.0  1.0  5.0  40.0      0.0     NaN  
3  14.0  15.0  1.0  1.0  3.0  0.0  4.0  40.0      0.0     NaN  
4   7.0  11.0  7.0  2.0  1.0  4.0  5.0  39.0      0.0     NaN  

[5 rows x 32 columns] Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', '

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [6]:
szn = season_stats[['Player', 'Age', 'Team', 'Pos', 'MP', 'FG', 'FGA',
       '3P', 'FT', 'FTA', 'TRB', 'AST', 'STL', 'BLK', 'PTS']]
szn

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Player,Age,Team,Pos,MP,FG,FGA,3P,FT,FTA,TRB,AST,STL,BLK,PTS
0,Luka Dončić,26.0,LAL,PG,41.0,17.0,27.0,2.0,7.0,10.0,12.0,9.0,2.0,1.0,43.0
1,Anthony Edwards,24.0,MIN,SG,39.0,14.0,28.0,5.0,8.0,8.0,7.0,1.0,0.0,1.0,41.0
2,Tyrese Maxey,25.0,PHI,PG,41.0,13.0,24.0,7.0,7.0,8.0,2.0,6.0,2.0,1.0,40.0
3,Victor Wembanyama,22.0,SAS,C,30.0,15.0,21.0,1.0,9.0,11.0,15.0,1.0,1.0,3.0,40.0
4,Alperen Şengün,23.0,HOU,C,49.0,12.0,24.0,5.0,10.0,11.0,11.0,7.0,2.0,1.0,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,Brandon Williams,26.0,DAL,PG,12.0,0.0,1.0,0.0,0.0,2.0,0.0,2.0,1.0,0.0,0.0
307,Cody Williams,21.0,UTA,SG,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
308,Guerschon Yabusele,30.0,NYK,C,12.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
309,Chris Youngblood,23.0,OKC,SG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
totals = fantasydf.merge(szn, how='left', on='Player')
totals

,Team_x,Player,Age,Team_y,Pos,MP,FG,FGA,3P,FT,FTA,TRB,AST,STL,BLK,PTS
0,Geriatric Knee's and Buckets,Cade Cunningham,24.0,DET,PG,34.0,8.0,24.0,1.0,6.0,10.0,7.0,10.0,1.0,0.0,23.0
1,Geriatric Knee's and Buckets,Josh Giddey,23.0,CHI,PG,29.0,5.0,12.0,1.0,8.0,11.0,5.0,11.0,0.0,1.0,19.0
2,Geriatric Knee's and Buckets,Kevin Durant,37.0,HOU,PF,47.0,9.0,16.0,0.0,5.0,6.0,9.0,3.0,0.0,0.0,23.0
3,Geriatric Knee's and Buckets,Chet Holmgren,23.0,OKC,C,39.0,11.0,17.0,2.0,4.0,4.0,7.0,2.0,0.0,0.0,28.0
4,Geriatric Knee's and Buckets,Jaren Jackson Jr.,26.0,MEM,C,27.0,7.0,14.0,0.0,4.0,4.0,8.0,1.0,0.0,6.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Dev,D'Angelo Russell,29.0,DAL,PG,15.0,1.0,6.0,0.0,4.0,6.0,2.0,3.0,0.0,0.0,6.0
104,Dev,Nic Claxton,26.0,BRK,C,23.0,8.0,12.0,0.0,1.0,2.0,4.0,2.0,0.0,0.0,17.0
105,Dev,Collin Sexton,27.0,CHO,SG,17.0,4.0,9.0,1.0,6.0,7.0,1.0,3.0,1.0,0.0,15.0
106,Dev,Davion Mitchell,27.0,MIA,PG,35.0,7.0,15.0,2.0,0.0,0.0,6.0,12.0,2.0,1.0,16.0


In [8]:
!pip install rapidfuzz
from rapidfuzz import process, fuzz
fantasydf['Player_clean'] = fantasydf['Player'].str.replace(r'\*', '', regex=True).str.lower()
szn['Player_clean'] = szn['Player'].str.replace(r'\*', '', regex=True).str.lower()

# Function to find the closest match from szn for a given fantasy player
def get_best_match(name, choices, score_cutoff=80):
    match = process.extractOne(name, choices, scorer=fuzz.token_sort_ratio)
    if match and match[1] >= score_cutoff:
        return match[0], match[1]
    else:
        return None, None

# Apply mapping
matches = []
for player in fantasydf['Player_clean']:
    best_match, score = get_best_match(player, szn['Player_clean'])
    matches.append({'Fantasy_Player': player, 'Best_Szn_Match': best_match, 'Score': score})

matches_df = pd.DataFrame(matches)

# Players with no good match
unmatched = matches_df[matches_df['Best_Szn_Match'].isna()]
print("Unmatched players:\n", unmatched)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 51.8 MB/s eta 0:00:00
Unmatched players:
          Fantasy_Player Best_Szn_Match  Score
12      christian braun           None    NaN
22         de'aaron fox           None    NaN
24          paul george           None    NaN
37      andrew nembhard           None    NaN
72         lebron james           None    NaN
76       darius garland           None    NaN
78   bennedict mathurin           None    NaN
79      cameron johnson           None    NaN
80         aaron gordon           None    NaN
86     domantas sabonis           None    NaN
88         jamal murray           None    NaN
90          tyler herro           None    NaN
95        pascal siakam           None    NaN
102          coby white           None    NaN


/tmp/ipykernel_13/3212877986.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  szn['Player_clean'] = szn['Player'].str.replace(r'\*', '', regex=True).str.lower()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [9]:
matches

[{'Fantasy_Player': 'cade cunningham',
  'Best_Szn_Match': 'cade cunningham',
  'Score': 100.0},
 {'Fantasy_Player': 'josh giddey',
  'Best_Szn_Match': 'josh giddey',
  'Score': 100.0},
 {'Fantasy_Player': 'kevin durant',
  'Best_Szn_Match': 'kevin durant',
  'Score': 100.0},
 {'Fantasy_Player': 'chet holmgren',
  'Best_Szn_Match': 'chet holmgren',
  'Score': 100.0},
 {'Fantasy_Player': 'jaren jackson jr.',
  'Best_Szn_Match': 'jaren jackson jr.',
  'Score': 100.0},
 {'Fantasy_Player': 'jimmy butler iii',
  'Best_Szn_Match': 'jimmy butler',
  'Score': 85.71428571428572},
 {'Fantasy_Player': 'draymond green',
  'Best_Szn_Match': 'draymond green',
  'Score': 100.0},
 {'Fantasy_Player': 'kawhi leonard',
  'Best_Szn_Match': 'kawhi leonard',
  'Score': 100.0},
 {'Fantasy_Player': 'rj barrett',
  'Best_Szn_Match': 'rj barrett',
  'Score': 100.0},
 {'Fantasy_Player': "kel'el ware",
  'Best_Szn_Match': "kel'el ware",
  'Score': 100.0},
 {'Fantasy_Player': 'cam thomas',
  'Best_Szn_Match': 'cam

In [10]:
fantasydf1 = fantasydf.merge(matches_df, left_on='Player_clean', right_on='Fantasy_Player', how='left')
print(fantasydf1.head())

                           Team             Player       Player_clean  \
0  Geriatric Knee's and Buckets    Cade Cunningham    cade cunningham   
1  Geriatric Knee's and Buckets        Josh Giddey        josh giddey   
2  Geriatric Knee's and Buckets       Kevin Durant       kevin durant   
3  Geriatric Knee's and Buckets      Chet Holmgren      chet holmgren   
4  Geriatric Knee's and Buckets  Jaren Jackson Jr.  jaren jackson jr.   

      Fantasy_Player     Best_Szn_Match  Score  
0    cade cunningham    cade cunningham  100.0  
1        josh giddey        josh giddey  100.0  
2       kevin durant       kevin durant  100.0  
3      chet holmgren      chet holmgren  100.0  
4  jaren jackson jr.  jaren jackson jr.  100.0  


In [11]:
fantasyszn = fantasydf1.merge(szn, left_on='Best_Szn_Match', right_on='Player_clean', how='left')
print(fantasyszn.head(), fantasyszn.columns)

                         Team_x           Player_x     Player_clean_x  \
0  Geriatric Knee's and Buckets    Cade Cunningham    cade cunningham   
1  Geriatric Knee's and Buckets        Josh Giddey        josh giddey   
2  Geriatric Knee's and Buckets       Kevin Durant       kevin durant   
3  Geriatric Knee's and Buckets      Chet Holmgren      chet holmgren   
4  Geriatric Knee's and Buckets  Jaren Jackson Jr.  jaren jackson jr.   

      Fantasy_Player     Best_Szn_Match  Score           Player_y   Age  \
0    cade cunningham    cade cunningham  100.0    Cade Cunningham  24.0   
1        josh giddey        josh giddey  100.0        Josh Giddey  23.0   
2       kevin durant       kevin durant  100.0       Kevin Durant  37.0   
3      chet holmgren      chet holmgren  100.0      Chet Holmgren  23.0   
4  jaren jackson jr.  jaren jackson jr.  100.0  Jaren Jackson Jr.  26.0   

  Team_y Pos  ...   FGA   3P   FT   FTA  TRB   AST  STL  BLK   PTS  \
0    DET  PG  ...  24.0  1.0  6.0  10.0 

In [12]:
fantasystats = fantasyszn[['Team_x', 'Fantasy_Player', 'Age', 'Team_y', 'Pos', 'FG',
       'FGA', '3P', 'FT', 'FTA', 'TRB', 'AST', 'STL', 'BLK', 'PTS']]
fantasystats

,Team_x,Fantasy_Player,Age,Team_y,Pos,FG,FGA,3P,FT,FTA,TRB,AST,STL,BLK,PTS
0,Geriatric Knee's and Buckets,cade cunningham,24.0,DET,PG,8.0,24.0,1.0,6.0,10.0,7.0,10.0,1.0,0.0,23.0
1,Geriatric Knee's and Buckets,josh giddey,23.0,CHI,PG,5.0,12.0,1.0,8.0,11.0,5.0,11.0,0.0,1.0,19.0
2,Geriatric Knee's and Buckets,kevin durant,37.0,HOU,PF,9.0,16.0,0.0,5.0,6.0,9.0,3.0,0.0,0.0,23.0
3,Geriatric Knee's and Buckets,chet holmgren,23.0,OKC,C,11.0,17.0,2.0,4.0,4.0,7.0,2.0,0.0,0.0,28.0
4,Geriatric Knee's and Buckets,jaren jackson jr.,26.0,MEM,C,7.0,14.0,0.0,4.0,4.0,8.0,1.0,0.0,6.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Dev,d'angelo russell,29.0,DAL,PG,1.0,6.0,0.0,4.0,6.0,2.0,3.0,0.0,0.0,6.0
104,Dev,nic claxton,26.0,BRK,C,8.0,12.0,0.0,1.0,2.0,4.0,2.0,0.0,0.0,17.0
105,Dev,collin sexton,27.0,CHO,SG,4.0,9.0,1.0,6.0,7.0,1.0,3.0,1.0,0.0,15.0
106,Dev,davion mitchell,27.0,MIA,PG,7.0,15.0,2.0,0.0,0.0,6.0,12.0,2.0,1.0,16.0


In [13]:
team_agg = fantasystats.groupby('Team_x').mean(numeric_only=True).round(2)
team_agg

,Age,FG,FGA,3P,FT,FTA,TRB,AST,STL,BLK,PTS
Team_x,,,,,,,,,,,
Belt,26.20,6.70,14.90,1.50,4.50,6.50,5.20,4.20,1.20,0.80,19.40
Da Man Dem,27.33,7.42,15.08,1.50,2.83,4.67,5.75,4.33,1.25,0.50,19.17
Dade County Snipers,23.85,7.92,15.62,1.62,4.92,6.38,6.31,5.08,1.00,1.15,22.38
Dev,27.33,5.42,12.08,1.33,3.67,4.75,5.00,4.17,0.92,0.33,15.83
Geriatric Knee's and Buckets,27.58,6.00,12.17,0.83,5.33,6.50,6.17,4.67,0.67,1.08,18.17
Real Njggas do Real Things,27.92,6.46,14.69,1.85,4.15,4.62,6.23,2.85,0.77,0.54,18.92
Teague Time,27.00,8.50,15.90,2.10,5.30,6.90,8.00,2.70,1.20,1.30,24.40
TytheGreat,26.25,6.92,14.00,1.75,2.42,3.25,5.33,3.17,1.00,0.58,18.00


In [14]:
week1 = fantasystats.copy()

week1 = week1.drop(columns=['Pos', 'Age'])
week1 = week1.rename(columns={'Fantasy_Player': 'Player', 'Team_x': 'Fantasy_Team', 'Team_y': 'NBA_Team'})

week1 = week1[['Fantasy_Team', 'Player', 'NBA_Team', 'FG', 'FGA', '3P', 'FT', 'FTA', 'TRB', 'AST', 'STL', 'BLK', 'PTS']]

print(week1.head(), week1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Fantasy_Team  108 non-null    object 
 1   Player        108 non-null    object 
 2   NBA_Team      94 non-null     object 
 3   FG            94 non-null     float64
 4   FGA           94 non-null     float64
 5   3P            94 non-null     float64
 6   FT            94 non-null     float64
 7   FTA           94 non-null     float64
 8   TRB           94 non-null     float64
 9   AST           94 non-null     float64
 10  STL           94 non-null     float64
 11  BLK           94 non-null     float64
 12  PTS           94 non-null     float64
dtypes: float64(10), object(3)
memory usage: 11.1+ KB
                   Fantasy_Team             Player NBA_Team    FG   FGA   3P  \
0  Geriatric Knee's and Buckets    cade cunningham      DET   8.0  24.0  1.0   
1  Geriatric Knee's and Buckets     

In [15]:
week1

,Fantasy_Team,Player,NBA_Team,FG,FGA,3P,FT,FTA,TRB,AST,STL,BLK,PTS
0,Geriatric Knee's and Buckets,cade cunningham,DET,8.0,24.0,1.0,6.0,10.0,7.0,10.0,1.0,0.0,23.0
1,Geriatric Knee's and Buckets,josh giddey,CHI,5.0,12.0,1.0,8.0,11.0,5.0,11.0,0.0,1.0,19.0
2,Geriatric Knee's and Buckets,kevin durant,HOU,9.0,16.0,0.0,5.0,6.0,9.0,3.0,0.0,0.0,23.0
3,Geriatric Knee's and Buckets,chet holmgren,OKC,11.0,17.0,2.0,4.0,4.0,7.0,2.0,0.0,0.0,28.0
4,Geriatric Knee's and Buckets,jaren jackson jr.,MEM,7.0,14.0,0.0,4.0,4.0,8.0,1.0,0.0,6.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Dev,d'angelo russell,DAL,1.0,6.0,0.0,4.0,6.0,2.0,3.0,0.0,0.0,6.0
104,Dev,nic claxton,BRK,8.0,12.0,0.0,1.0,2.0,4.0,2.0,0.0,0.0,17.0
105,Dev,collin sexton,CHO,4.0,9.0,1.0,6.0,7.0,1.0,3.0,1.0,0.0,15.0
106,Dev,davion mitchell,MIA,7.0,15.0,2.0,0.0,0.0,6.0,12.0,2.0,1.0,16.0


In [16]:
topweek1 = week1.groupby('Fantasy_Team').sum(numeric_only=True).round(2)
topweek1['FG%'] = topweek1['FG'] / topweek1['FGA']
topweek1['FT%'] = topweek1['FT'] / topweek1['FTA']
print(topweek1.head())

                                 FG    FGA    3P    FT   FTA   TRB   AST  \
Fantasy_Team                                                               
Belt                           67.0  149.0  15.0  45.0  65.0  52.0  42.0   
Da Man Dem                     89.0  181.0  18.0  34.0  56.0  69.0  52.0   
Dade County Snipers           103.0  203.0  21.0  64.0  83.0  82.0  66.0   
Dev                            65.0  145.0  16.0  44.0  57.0  60.0  50.0   
Geriatric Knee's and Buckets   72.0  146.0  10.0  64.0  78.0  74.0  56.0   

                               STL   BLK    PTS       FG%       FT%  
Fantasy_Team                                                         
Belt                          12.0   8.0  194.0  0.449664  0.692308  
Da Man Dem                    15.0   6.0  230.0  0.491713  0.607143  
Dade County Snipers           13.0  15.0  291.0  0.507389  0.771084  
Dev                           11.0   4.0  190.0  0.448276  0.771930  
Geriatric Knee's and Buckets   8.0  13.0  218.0

In [17]:
rank_df = topweek1.copy()

categories = ['PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%', 'FT%', '3P']

for cat in categories:
    rank_df[f'{cat}_Rank'] = rank_df[cat].rank(ascending=False, method='min')

rank_df['Sum'] = rank_df[[f'{cat}_Rank' for cat in categories]].sum(axis=1)

rank_df = rank_df.sort_values('Sum')
rankedwk1 = rank_df[['Sum', 'PTS', 'TRB', 'AST', 'STL', 'BLK', '3P', 'FG%', 'FT%']]
rankedwk1

,Sum,PTS,TRB,AST,STL,BLK,3P,FG%,FT%
Fantasy_Team,,,,,,,,,
Dade County Snipers,14.0,291.0,82.0,66.0,13.0,15.0,21.0,0.507389,0.771084
Teague Time,27.0,244.0,80.0,27.0,12.0,13.0,21.0,0.534591,0.768116
Real Njggas do Real Things,33.0,246.0,81.0,37.0,10.0,7.0,24.0,0.439791,0.900000
Geriatric Knee's and Buckets,35.0,218.0,74.0,56.0,8.0,13.0,10.0,0.493151,0.820513
TytheGreat,37.0,216.0,64.0,38.0,12.0,7.0,21.0,0.494048,0.743590
Da Man Dem,38.0,230.0,69.0,52.0,15.0,6.0,18.0,0.491713,0.607143
Belt,47.0,194.0,52.0,42.0,12.0,8.0,15.0,0.449664,0.692308
Dev,49.0,190.0,60.0,50.0,11.0,4.0,16.0,0.448276,0.771930
